In [1]:
#@title # 1. Setup and Imports
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import random

# Set up the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a seed for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

# --- Experiment Configuration ---
# This value simulates the standard deviation of Gaussian noise added after each
# HE-emulated operation. It's a crucial parameter to tune. Based on theoretical
# analysis of CKKS, the noise is small relative to the message scale. For data
# normalized to [0, 1], a small sigma is appropriate.
HE_NOISE_SIGMA = 1e-4
EPOCHS = 10 # Number of epochs to train each model configuration
LEARNING_RATE = 0.001
BATCH_SIZE = 64

Using device: cuda


In [2]:
#@title # 2. HE Noise Simulation Module
class SimulatedHENoise(nn.Module):
    """
    A module to inject Gaussian noise, simulating the noise from HE operations.
    Noise is only added during training.
    """
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, x):
        # Only inject noise during training and if sigma is positive
        if self.training and self.sigma > 0:
            noise = torch.randn_like(x) * self.sigma
            return x + noise
        return x

In [3]:
#@title # 3. Model Definitions with Noise Injection
# ==============================================================================
# Base class for models to share the noise injection logic
# ==============================================================================
class SplitModelBase(nn.Module):
    def __init__(self, cut_layer, he_noise_sigma):
        super().__init__()
        # The cut_layer index indicates the last layer on the server.
        # e.g., cut_layer=2 means layers at index 0 and 1 are on the server.
        self.cut_layer = cut_layer
        self.noise_injector = SimulatedHENoise(he_noise_sigma)
        self.layers_list = nn.ModuleList() # Subclasses will populate this

    def forward(self, x):
        for i, layer in enumerate(self.layers_list):
            x = layer(x)
            # If the layer is on the server (i.e., before or at the cut), inject noise
            if i < self.cut_layer:
                x = self.noise_injector(x)
        return x

# ==============================================================================
# 3.1. MNIST Fully Connected Network (784 -> 128 -> 32 -> 10)
# ==============================================================================
class MNIST_FCNN(SplitModelBase):
    def __init__(self, cut_layer, he_noise_sigma=HE_NOISE_SIGMA):
        super().__init__(cut_layer, he_noise_sigma)
        # Total layers: 5 (3 Linear, 2 ReLU)
        # Cut positions can be from 1 to 5
        self.layers_list.extend([
            nn.Flatten(),                     # Layer 0
            nn.Linear(28 * 28, 128),          # Layer 1
            nn.ReLU(),                        # Layer 2 (Cut after this is cut_layer=3)
            nn.Linear(128, 32),               # Layer 3
            nn.ReLU(),                        # Layer 4
            nn.Linear(32, 10)                 # Layer 5
        ])
        # Note: The effective number of "computation" blocks is 3 (Lin+Act, Lin+Act, Lin).
        # We allow cutting after any layer for fine-grained analysis.

    def forward(self, x):
        # The forward pass is slightly different due to Flatten
        for i, layer in enumerate(self.layers_list):
            x = layer(x)
            # Inject noise after Linear and ReLU layers on the server side
            if i > 0 and i < self.cut_layer: # Don't add noise after Flatten
                x = self.noise_injector(x)
        return x


# ==============================================================================
# 3.2. BCWFS Fully Connected Network (64 -> 32 -> 16 -> 10)
# ==============================================================================
class BCWFS_FCNN(SplitModelBase):
    def __init__(self, cut_layer, he_noise_sigma=HE_NOISE_SIGMA):
        super().__init__(cut_layer, he_noise_sigma)
        self.layers_list.extend([
            nn.Linear(64, 32),      # Layer 0
            nn.ReLU(),              # Layer 1
            nn.Linear(32, 16),      # Layer 2
            nn.ReLU(),              # Layer 3
            nn.Linear(16, 10)       # Layer 4
        ])


# ==============================================================================
# 3.3. PTB XL Convolutional Network
# ==============================================================================
class PTB_XL_CNN(SplitModelBase):
    def __init__(self, cut_layer, he_noise_sigma=HE_NOISE_SIGMA):
        super().__init__(cut_layer, he_noise_sigma)
        self.layers_list.extend([
            nn.Conv1d(in_channels=12, out_channels=16, kernel_size=3, padding=1), # Layer 0
            nn.LeakyReLU(),                                                       # Layer 1
            nn.MaxPool1d(kernel_size=2, stride=2),                                # Layer 2
            nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, padding=1),  # Layer 3
            nn.LeakyReLU(),                                                       # Layer 4
            nn.MaxPool1d(kernel_size=2, stride=2),                                # Layer 5
            nn.Flatten(),                                                         # Layer 6
            nn.Linear(8 * 250, 2000),                                             # Layer 7
            nn.Linear(2000, 5)                                                    # Layer 8
            # Softmax is handled by the loss function (nn.CrossEntropyLoss)
        ])


# ==============================================================================
# 3.4. LeNet for MNIST
# Ref: https://d2l.ai/chapter_convolutional-neural-networks/lenet.html
# ==============================================================================
class LeNet(SplitModelBase):
    def __init__(self, cut_layer, he_noise_sigma=HE_NOISE_SIGMA):
        super().__init__(cut_layer, he_noise_sigma)
        self.layers_list.extend([
            nn.Conv2d(1, 6, kernel_size=5, padding=2), # Layer 0
            nn.ReLU(),                                 # Layer 1
            nn.AvgPool2d(kernel_size=2, stride=2),     # Layer 2
            nn.Conv2d(6, 16, kernel_size=5),           # Layer 3
            nn.ReLU(),                                 # Layer 4
            nn.AvgPool2d(kernel_size=2, stride=2),     # Layer 5
            nn.Flatten(),                              # Layer 6
            nn.Linear(16 * 5 * 5, 120),                # Layer 7
            nn.ReLU(),                                 # Layer 8
            nn.Linear(120, 84),                        # Layer 9
            nn.ReLU(),                                 # Layer 10
            nn.Linear(84, 10)                          # Layer 11
        ])


In [29]:
#@title # 4. Data Loading and Preprocessing (Real Datasets)

# Required imports for data loading and preprocessing
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import requests
import zipfile
import ast # Added for PTB-XL literal_eval
import os # Make sure os is imported


# ==============================================================================
# 4.1. MNIST Data (No changes needed)
# ==============================================================================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_mnist = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader_mnist = DataLoader(train_mnist, batch_size=BATCH_SIZE, shuffle=True)
test_loader_mnist = DataLoader(test_mnist, batch_size=BATCH_SIZE, shuffle=False)

print(f"MNIST dataset loaded.")
print(f"Train samples: {len(train_mnist)}, Test samples: {len(test_mnist)}")

# ==============================================================================
# 4.2. Breast Cancer Wisconsin (Diagnostic) Dataset
# This function loads the standard BCW dataset, scales it, and pads features
# to match the model architecture (30 features -> 64 features).
# ==============================================================================
def get_bcw_loaders(batch_size=BATCH_SIZE):
    print("\n--- Loading real Breast Cancer Wisconsin dataset. ---")
    data = load_breast_cancer()
    X, y = data.data, data.target

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # --- Padding to match the model's 64-feature input ---
    # The original dataset has 30 features. We pad with zeros.
    X_train_padded = np.zeros((X_train_scaled.shape[0], 64))
    X_train_padded[:, :30] = X_train_scaled
    X_test_padded = np.zeros((X_test_scaled.shape[0], 64))
    X_test_padded[:, :30] = X_test_scaled
    print(f"BCW features padded from 30 to {X_train_padded.shape[1]}.")

    # Convert to PyTorch Tensors
    X_train_tensor = torch.tensor(X_train_padded, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test_padded, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Create TensorDatasets and DataLoaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

train_loader_bcwfs, test_loader_bcwfs = get_bcw_loaders()
print("Breast Cancer Wisconsin dataset loaded and preprocessed.")

# ==============================================================================
# 4.3. PTB-XL ECG Dataset
# This function loads and preprocesses the PTB-XL dataset. It now takes the
# path to the downloaded dataset files as an argument.
# ==============================================================================
def get_ptbxl_loaders(dataset_path, batch_size=BATCH_SIZE):
    print("\n--- Loading real PTB-XL dataset from downloaded path. ---")
    # Inspect the contents of the downloaded directory
    print(f"Contents of {dataset_path}: {os.listdir(dataset_path)}")

    db_file = os.path.join(dataset_path, 'ptbxl_database.csv')
    scp_file = os.path.join(dataset_path, 'scp_statements.csv')
    records_dir = os.path.join(dataset_path, 'records100') # Assuming records100 is a subdir


    # Check if files exist at the provided path
    if not os.path.exists(db_file):
        print(f"Error: Database file not found at {db_file}")
        return None, None
    if not os.path.exists(scp_file):
        print(f"Error: SCP statements file not found at {scp_file}")
        return None, None
    if not os.path.exists(records_dir):
        print(f"Error: Records directory not found at {records_dir}")
        return None, None

    print(f"Metadata files and records directory found at {dataset_path}.")

    # Load metadata and map to superclasses
    try:
        Y = pd.read_csv(db_file, index_col='ecg_id')
        Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
        scp_statements = pd.read_csv(scp_file, index_col=0)
        scp_statements = scp_statements[scp_statements.diagnostic == 1]

        def aggregate_diagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in scp_statements.index:
                    tmp.append(scp_statements.loc[key].diagnostic_class)
                elif key.replace("(", "").replace(")", "").replace("-", "") in scp_statements.index: # Try cleaning up key
                     tmp.append(scp_statements.loc[key.replace("(", "").replace(")", "").replace("-", "")].diagnostic_class)
            return list(set(tmp))

        Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
        print("Metadata loaded and processed.")
    except Exception as e:
        print(f"Error loading or processing metadata: {e}")
        return None, None


    # --- Use the dataset's recommended 10-fold split ---
    test_fold = 10
    train_folds = [1, 2, 3, 4, 5, 6, 7, 8, 9]

    try:
        # Train set
        train_filenames = Y[Y.strat_fold.isin(train_folds)].filename_lr.tolist()
        X_train = np.array([np.load(os.path.join(records_dir, f'{f}.npy')) for f in train_filenames])
        y_train = np.array([l[0] for l in Y[Y.strat_fold.isin(train_folds)].diagnostic_superclass]) # Take first label for multiclass
        print(f"Loaded {len(train_filenames)} training records.")

        # Test set
        test_filenames = Y[Y.strat_fold == test_fold].filename_lr.tolist()
        X_test = np.array([np.load(os.path.join(records_dir, f'{f}.npy')) for f in test_filenames])
        y_test = np.array([l[0] for l in Y[Y.strat_fold == test_fold].diagnostic_superclass]) # Take first label for multiclass
        print(f"Loaded {len(test_filenames)} testing records.")

    except Exception as e:
        print(f"Error loading .npy files: {e}")
        return None, None


    # --- Preprocessing ---
    # Transpose to (Channels, Length)
    try:
        X_train = X_train.transpose(0, 2, 1)
        X_test = X_test.transpose(0, 2, 1)
        print(f"PTB-XL data reshaped to {X_train.shape}.")

        # Label encoding from strings to integers
        superclass_map = {label: i for i, label in enumerate(np.unique(np.concatenate((y_train, y_test))))}
        y_train = np.array([superclass_map[label] for label in y_train])
        y_test = np.array([superclass_map[label] for label in y_test])
        print(f"PTB-XL labels mapped to {len(superclass_map)} classes.")
        print(f"X_train shape after preprocessing: {X_train.shape}")
        print(f"y_train shape after preprocessing: {y_train.shape}")
        print(f"X_test shape after preprocessing: {X_test.shape}")
        print(f"y_test shape after preprocessing: {y_test.shape}")


    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None, None


    # Convert to PyTorch Tensors
    try:
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)
        print("Converted data to PyTorch tensors.")

    except Exception as e:
        print(f"Error converting to tensors: {e}")
        return None, None

    # Create TensorDatasets and DataLoaders
    try:
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        print("Created DataLoaders.")
        print(f"train_loader type: {type(train_loader)}")
        print(f"test_loader type: {type(test_loader)}")

    except Exception as e:
        print(f"Error creating DataLoaders: {e}")
        return None, None


    return train_loader, test_loader

# Note: The PTB-XL download and preprocessing might take a minute or two the first time.
import ast
train_loader_ptbxl, test_loader_ptbxl = get_ptbxl_loaders('/root/.cache/kagglehub/datasets/khyeh0719/ptb-xl-dataset/versions/1')
print("PTB-XL dataset loaded and preprocessed.")

MNIST dataset loaded.
Train samples: 60000, Test samples: 10000

--- Loading real Breast Cancer Wisconsin dataset. ---
BCW features padded from 30 to 64.
Breast Cancer Wisconsin dataset loaded and preprocessed.

--- Loading real PTB-XL dataset from downloaded path. ---
Contents of /root/.cache/kagglehub/datasets/khyeh0719/ptb-xl-dataset/versions/1: ['ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1']
Error: Database file not found at /root/.cache/kagglehub/datasets/khyeh0719/ptb-xl-dataset/versions/1/ptbxl_database.csv
PTB-XL dataset loaded and preprocessed.


In [9]:
#@title # 5. Training and Evaluation Loops

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    """Trains the model for one epoch."""
    model.train()
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate(model, dataloader, device):
    """Evaluates the model and returns accuracy."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs) # Corrected: use inputs here
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [6]:

#@title # 6. ResNet Block Noise Analysis
def analyze_resnet_block_noise(initial_std_dev=1.0, op_sigma=HE_NOISE_SIGMA):
    """
    Simulates the accumulation of noise variance through a ResNet basic block.
    A ResNet block consists of two 3x3 convolutions (each followed by a simulated
    batch norm) and a skip connection.
    Ref: https://d2l.ai/chapter_convolutional-neural-networks/resnet.html [7]
    """
    print("\n" + "="*50)
    print("Analyzing Noise Accumulation in a ResNet Block")
    print(f"Noise sigma per HE operation: {op_sigma:.2e}")
    print(f"Initial input noise std dev: {initial_std_dev:.2e}")

    # Variances add. std_dev = sqrt(variance)
    # Start with initial variance
    identity_variance = initial_std_dev**2
    x_variance = identity_variance

    # --- Path through conv layers ---
    # 1. First Conv + BatchNorm
    # A convolution is a sum of products. In HE, this involves many multiplications
    # and rotations, each adding key-switching noise. We simplify this by adding
    # one noise term for the Conv op and one for the BatchNorm op.
    x_variance += op_sigma**2  # Noise from Conv1
    x_variance += op_sigma**2  # Noise from BatchNorm1
    # ReLU is a comparison, which is simpler in HE, we assume its noise is negligible.

    # 2. Second Conv + BatchNorm
    x_variance += op_sigma**2  # Noise from Conv2
    x_variance += op_sigma**2  # Noise from BatchNorm2

    # --- Skip Connection ---
    # The final step is F(x) + x. In HE, adding two ciphertexts sums their noise.
    final_variance = x_variance + identity_variance
    final_std_dev = np.sqrt(final_variance)

    print(f"Variance after conv path: {x_variance:.2e}")
    print(f"Variance after skip connection add: {final_variance:.2e}")
    print(f"Final noise std dev after one block: {final_std_dev:.2e}")
    print("="*50 + "\n")

    return {
        'model': 'ResNet_Block_Analysis',
        'cut_layer': 'N/A',
        'accuracy': 'N/A',
        'final_noise_stddev': final_std_dev
    }


In [10]:


#@title # 7. Experiment Orchestration
def run_experiments():
    """
    Main function to run all experiments, collect results, and save to CSV.
    """
    results = []

    # Define all models, their data, and the number of layers to test cuts for
    models_to_test = {
        "MNIST_FCNN": {
            "class": MNIST_FCNN,
            "train_loader": train_loader_mnist,
            "test_loader": test_loader_mnist,
            "total_layers": 6 # 0-5
        },
        "BCWFS_FCNN": {
            "class": BCWFS_FCNN,
            "train_loader": train_loader_bcwfs,
            "test_loader": test_loader_bcwfs,
            "total_layers": 5 # 0-4
        },
        "PTB_XL_CNN": {
            "class": PTB_XL_CNN,
            "train_loader": train_loader_ptbxl,
            "test_loader": test_loader_ptbxl,
            "total_layers": 9 # 0-8
        },
        "LeNet": {
            "class": LeNet,
            "train_loader": train_loader_mnist,
            "test_loader": test_loader_mnist,
            "total_layers": 12 # 0-11
        }
    }

    # Loss function
    criterion = nn.CrossEntropyLoss()

    for name, config in models_to_test.items():
        # Iterate through all possible cut layer positions, including 0 (all client)
        # and total_layers (all server)
        for cut in tqdm(range(config['total_layers'] + 1), desc=f"Testing {name} cuts"):
            print(f"\n--- Training {name} with cut_layer = {cut} ---")
            print(f"Server-side layers: 0 to {cut-1}")

            # Instantiate model
            model = config['class'](cut_layer=cut, he_noise_sigma=HE_NOISE_SIGMA).to(device)
            optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

            # Training loop
            for epoch in range(EPOCHS):
                train_one_epoch(model, config['train_loader'], criterion, optimizer, device)
                acc = evaluate(model, config['test_loader'], device)
                print(f"Epoch {epoch+1}/{EPOCHS}, Test Accuracy: {acc:.2f}%")

            # Final evaluation
            final_accuracy = evaluate(model, config['test_loader'], device)
            print(f"Final Accuracy for {name} (cut={cut}): {final_accuracy:.2f}%")

            # Store result
            results.append({
                'model': name,
                'cut_layer': cut,
                'accuracy': final_accuracy,
                'final_noise_stddev': 'N/A'
            })

    # Run ResNet noise analysis
    resnet_result = analyze_resnet_block_noise()
    results.append(resnet_result)

    return pd.DataFrame(results)

# Run all experiments
results_df = run_experiments()

#@title # 8. Save and Display Results
print("\n\n--- All Experiments Complete ---")
print("Final Accuracy and Noise Analysis Results:")

# Display the results DataFrame
display(results_df)

# Save the DataFrame to a CSV file
output_filename = 'split_learning_accuracy_results.csv'
results_df.to_csv(output_filename, index=False)
print(f"\nResults saved to {output_filename}")

Testing MNIST_FCNN cuts:   0%|          | 0/7 [00:00<?, ?it/s]


--- Training MNIST_FCNN with cut_layer = 0 ---
Server-side layers: 0 to -1
Epoch 1/10, Test Accuracy: 92.98%
Epoch 2/10, Test Accuracy: 94.12%
Epoch 3/10, Test Accuracy: 95.81%
Epoch 4/10, Test Accuracy: 96.11%
Epoch 5/10, Test Accuracy: 96.74%
Epoch 6/10, Test Accuracy: 95.96%
Epoch 7/10, Test Accuracy: 96.46%
Epoch 8/10, Test Accuracy: 97.06%
Epoch 9/10, Test Accuracy: 96.41%
Epoch 10/10, Test Accuracy: 97.25%
Final Accuracy for MNIST_FCNN (cut=0): 97.25%

--- Training MNIST_FCNN with cut_layer = 1 ---
Server-side layers: 0 to 0
Epoch 1/10, Test Accuracy: 92.77%
Epoch 2/10, Test Accuracy: 93.67%
Epoch 3/10, Test Accuracy: 95.73%
Epoch 4/10, Test Accuracy: 95.56%
Epoch 5/10, Test Accuracy: 96.64%
Epoch 6/10, Test Accuracy: 96.99%
Epoch 7/10, Test Accuracy: 96.69%
Epoch 8/10, Test Accuracy: 96.75%
Epoch 9/10, Test Accuracy: 96.48%
Epoch 10/10, Test Accuracy: 97.03%
Final Accuracy for MNIST_FCNN (cut=1): 97.03%

--- Training MNIST_FCNN with cut_layer = 2 ---
Server-side layers: 0 to 1


Testing BCWFS_FCNN cuts:   0%|          | 0/6 [00:00<?, ?it/s]


--- Training BCWFS_FCNN with cut_layer = 0 ---
Server-side layers: 0 to -1
Epoch 1/10, Test Accuracy: 13.00%
Epoch 2/10, Test Accuracy: 12.50%
Epoch 3/10, Test Accuracy: 13.50%
Epoch 4/10, Test Accuracy: 14.00%
Epoch 5/10, Test Accuracy: 13.00%
Epoch 6/10, Test Accuracy: 12.00%
Epoch 7/10, Test Accuracy: 12.00%
Epoch 8/10, Test Accuracy: 11.50%
Epoch 9/10, Test Accuracy: 11.00%
Epoch 10/10, Test Accuracy: 12.00%
Final Accuracy for BCWFS_FCNN (cut=0): 12.00%

--- Training BCWFS_FCNN with cut_layer = 1 ---
Server-side layers: 0 to 0
Epoch 1/10, Test Accuracy: 9.50%
Epoch 2/10, Test Accuracy: 11.00%
Epoch 3/10, Test Accuracy: 11.00%
Epoch 4/10, Test Accuracy: 10.50%
Epoch 5/10, Test Accuracy: 10.50%
Epoch 6/10, Test Accuracy: 10.00%
Epoch 7/10, Test Accuracy: 10.50%
Epoch 8/10, Test Accuracy: 11.50%
Epoch 9/10, Test Accuracy: 12.00%
Epoch 10/10, Test Accuracy: 10.50%
Final Accuracy for BCWFS_FCNN (cut=1): 10.50%

--- Training BCWFS_FCNN with cut_layer = 2 ---
Server-side layers: 0 to 1
E

Testing PTB_XL_CNN cuts:   0%|          | 0/10 [00:00<?, ?it/s]


--- Training PTB_XL_CNN with cut_layer = 0 ---
Server-side layers: 0 to -1
Epoch 1/10, Test Accuracy: 21.00%
Epoch 2/10, Test Accuracy: 19.50%
Epoch 3/10, Test Accuracy: 22.00%
Epoch 4/10, Test Accuracy: 21.00%
Epoch 5/10, Test Accuracy: 23.00%
Epoch 6/10, Test Accuracy: 20.00%
Epoch 7/10, Test Accuracy: 22.50%
Epoch 8/10, Test Accuracy: 21.50%
Epoch 9/10, Test Accuracy: 21.50%
Epoch 10/10, Test Accuracy: 20.50%
Final Accuracy for PTB_XL_CNN (cut=0): 20.50%

--- Training PTB_XL_CNN with cut_layer = 1 ---
Server-side layers: 0 to 0
Epoch 1/10, Test Accuracy: 19.00%
Epoch 2/10, Test Accuracy: 20.50%
Epoch 3/10, Test Accuracy: 23.50%
Epoch 4/10, Test Accuracy: 22.50%
Epoch 5/10, Test Accuracy: 18.00%
Epoch 6/10, Test Accuracy: 19.50%
Epoch 7/10, Test Accuracy: 20.00%
Epoch 8/10, Test Accuracy: 17.50%
Epoch 9/10, Test Accuracy: 17.50%
Epoch 10/10, Test Accuracy: 20.50%
Final Accuracy for PTB_XL_CNN (cut=1): 20.50%

--- Training PTB_XL_CNN with cut_layer = 2 ---
Server-side layers: 0 to 1


Testing LeNet cuts:   0%|          | 0/13 [00:00<?, ?it/s]


--- Training LeNet with cut_layer = 0 ---
Server-side layers: 0 to -1
Epoch 1/10, Test Accuracy: 97.39%
Epoch 2/10, Test Accuracy: 98.12%
Epoch 3/10, Test Accuracy: 98.79%
Epoch 4/10, Test Accuracy: 98.84%
Epoch 5/10, Test Accuracy: 98.70%
Epoch 6/10, Test Accuracy: 98.73%
Epoch 7/10, Test Accuracy: 98.92%
Epoch 8/10, Test Accuracy: 98.93%
Epoch 9/10, Test Accuracy: 99.07%
Epoch 10/10, Test Accuracy: 98.95%
Final Accuracy for LeNet (cut=0): 98.95%

--- Training LeNet with cut_layer = 1 ---
Server-side layers: 0 to 0
Epoch 1/10, Test Accuracy: 97.37%
Epoch 2/10, Test Accuracy: 98.00%
Epoch 3/10, Test Accuracy: 98.78%
Epoch 4/10, Test Accuracy: 98.81%
Epoch 5/10, Test Accuracy: 98.91%
Epoch 6/10, Test Accuracy: 98.81%
Epoch 7/10, Test Accuracy: 98.93%
Epoch 8/10, Test Accuracy: 99.21%
Epoch 9/10, Test Accuracy: 98.87%
Epoch 10/10, Test Accuracy: 99.17%
Final Accuracy for LeNet (cut=1): 99.17%

--- Training LeNet with cut_layer = 2 ---
Server-side layers: 0 to 1
Epoch 1/10, Test Accuracy

,model,cut_layer,accuracy,final_noise_stddev
0,MNIST_FCNN,0,97.25,N/A
1,MNIST_FCNN,1,97.03,N/A
2,MNIST_FCNN,2,97.19,N/A
3,MNIST_FCNN,3,96.84,N/A
4,MNIST_FCNN,4,96.63,N/A
5,MNIST_FCNN,5,97.09,N/A
6,MNIST_FCNN,6,97.16,N/A
7,BCWFS_FCNN,0,12.0,N/A
8,BCWFS_FCNN,1,10.5,N/A
9,BCWFS_FCNN,2,11.5,N/A



Results saved to split_learning_accuracy_results.csv


In [37]:
#@title # 7. Experiment Orchestration
def run_experiments():
    """
    Main function to run all experiments, collect results, and save to CSV.
    """
    results = []

    # Define all models, their data, and the number of layers to test cuts for
    models_to_test = {
        "BCWFS_FCNN": {
            "class": BCWFS_FCNN,
            "train_loader": train_loader_bcwfs,
            "test_loader": test_loader_bcwfs,
            "total_layers": 5 # 0-4
        },
        "PTB_XL_CNN": {
            "class": PTB_XL_CNN,
            "train_loader": train_loader_ptbxl,
            "test_loader": test_loader_ptbxl,
            "total_layers": 9 # 0-8
        },

    }

    # Loss function
    criterion = nn.CrossEntropyLoss()

    for name, config in models_to_test.items():
        # Check if data loaders were successfully loaded
        if config['train_loader'] is None or config['test_loader'] is None:
            print(f"\n--- Skipping {name} experiments due to data loading failure. ---")
            continue

        # Iterate through all possible cut layer positions, including 0 (all client)
        # and total_layers (all server)
        for cut in tqdm(range(config['total_layers'] + 1), desc=f"Testing {name} cuts"):
            print(f"\n--- Training {name} with cut_layer = {cut} ---")
            print(f"Server-side layers: 0 to {cut-1}")

            # Instantiate model
            model = config['class'](cut_layer=cut, he_noise_sigma=HE_NOISE_SIGMA).to(device)
            optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

            # Training loop
            for epoch in range(EPOCHS):
                train_one_epoch(model, config['train_loader'], criterion, optimizer, device)
                acc = evaluate(model, config['test_loader'], device)
                print(f"Epoch {epoch+1}/{EPOCHS}, Test Accuracy: {acc:.2f}%")

            # Final evaluation
            final_accuracy = evaluate(model, config['test_loader'], device)
            print(f"Final Accuracy for {name} (cut={cut}): {final_accuracy:.2f}%")

            # Store result
            results.append({
                'model': name,
                'cut_layer': cut,
                'accuracy': final_accuracy,
                'final_noise_stddev': 'N/A'
            })

    # Run ResNet noise analysis
    resnet_result = analyze_resnet_block_noise()
    results.append(resnet_result)

    return pd.DataFrame(results)

# Run all experiments
results_df = run_experiments()

#@title # 8. Save and Display Results
print("\n\n--- All Experiments Complete ---")
print("Final Accuracy and Noise Analysis Results:")

# Display the results DataFrame
display(results_df)

# Save the DataFrame to a CSV file
output_filename = 'split_learning_accuracy_results.csv'
results_df.to_csv(output_filename, index=False)
print(f"\nResults saved to {output_filename}")

Testing BCWFS_FCNN cuts:   0%|          | 0/6 [00:00<?, ?it/s]


--- Training BCWFS_FCNN with cut_layer = 0 ---
Server-side layers: 0 to -1
Epoch 1/10, Test Accuracy: 0.00%
Epoch 2/10, Test Accuracy: 6.14%
Epoch 3/10, Test Accuracy: 41.23%
Epoch 4/10, Test Accuracy: 64.91%
Epoch 5/10, Test Accuracy: 83.33%
Epoch 6/10, Test Accuracy: 93.86%
Epoch 7/10, Test Accuracy: 94.74%
Epoch 8/10, Test Accuracy: 94.74%
Epoch 9/10, Test Accuracy: 94.74%
Epoch 10/10, Test Accuracy: 94.74%
Final Accuracy for BCWFS_FCNN (cut=0): 94.74%

--- Training BCWFS_FCNN with cut_layer = 1 ---
Server-side layers: 0 to 0
Epoch 1/10, Test Accuracy: 60.53%
Epoch 2/10, Test Accuracy: 88.60%
Epoch 3/10, Test Accuracy: 89.47%
Epoch 4/10, Test Accuracy: 91.23%
Epoch 5/10, Test Accuracy: 89.47%
Epoch 6/10, Test Accuracy: 88.60%
Epoch 7/10, Test Accuracy: 88.60%
Epoch 8/10, Test Accuracy: 89.47%
Epoch 9/10, Test Accuracy: 89.47%
Epoch 10/10, Test Accuracy: 89.47%
Final Accuracy for BCWFS_FCNN (cut=1): 89.47%

--- Training BCWFS_FCNN with cut_layer = 2 ---
Server-side layers: 0 to 1
Ep

,model,cut_layer,accuracy,final_noise_stddev
0,BCWFS_FCNN,0,94.736842,N/A
1,BCWFS_FCNN,1,89.473684,N/A
2,BCWFS_FCNN,2,93.859649,N/A
3,BCWFS_FCNN,3,90.350877,N/A
4,BCWFS_FCNN,4,90.350877,N/A
5,BCWFS_FCNN,5,91.22807,N/A
6,ResNet_Block_Analysis,N/A,N/A,1.414214



Results saved to split_learning_accuracy_results.csv


In [22]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("khyeh0719/ptb-xl-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.72G/1.72G [00:12<00:00, 150MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/khyeh0719/ptb-xl-dataset/versions/1


In [26]:
# Use the path from the kagglehub download
ptbxl_dataset_path = '/root/.cache/kagglehub/datasets/khyeh0719/ptb-xl-dataset/versions/1'
train_loader_ptbxl_only, test_loader_ptbxl_only = get_ptbxl_loaders(ptbxl_dataset_path)
print("PTB-XL dataset loaded and preprocessed for isolated experiment.")


--- Loading real PTB-XL dataset from downloaded path. ---
Error: Database file not found at /root/.cache/kagglehub/datasets/khyeh0719/ptb-xl-dataset/versions/1/ptbxl_database.csv
PTB-XL dataset loaded and preprocessed for isolated experiment.


In [30]:
#@title # 7. Experiment Orchestration (PTB-XL Only)
def run_ptbxl_experiments():
    """
    Runs experiments only for the PTB-XL CNN model.
    """
    results = []

    # Define only the PTB-XL model and its data
    models_to_test = {
        "PTB_XL_CNN": {
            "class": PTB_XL_CNN,
            "train_loader": train_loader_ptbxl_only, # Use the specifically loaded data
            "test_loader": test_loader_ptbxl_only,   # Use the specifically loaded data
            "total_layers": 9 # 0-8
        },
    }

    # Loss function
    criterion = nn.CrossEntropyLoss()

    for name, config in models_to_test.items():
        # Check if data loaders were successfully loaded
        if config['train_loader'] is None or config['test_loader'] is None:
            print(f"\n--- Skipping {name} experiments due to data loading failure. ---")
            continue

        # Iterate through all possible cut layer positions, including 0 (all client)
        # and total_layers (all server)
        for cut in tqdm(range(config['total_layers'] + 1), desc=f"Testing {name} cuts"):
            print(f"\n--- Training {name} with cut_layer = {cut} ---")
            print(f"Server-side layers: 0 to {cut-1}")

            # Instantiate model
            model = config['class'](cut_layer=cut, he_noise_sigma=HE_NOISE_SIGMA).to(device)
            optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

            # Training loop
            for epoch in range(EPOCHS):
                train_one_epoch(model, config['train_loader'], criterion, optimizer, device)
                acc = evaluate(model, config['test_loader'], device)
                print(f"Epoch {epoch+1}/{EPOCHS}, Test Accuracy: {acc:.2f}%")

            # Final evaluation
            final_accuracy = evaluate(model, config['test_loader'], device)
            print(f"Final Accuracy for {name} (cut={cut}): {final_accuracy:.2f}%")

            # Store result
            results.append({
                'model': name,
                'cut_layer': cut,
                'accuracy': final_accuracy,
                'final_noise_stddev': 'N/A'
            })

    # ResNet noise analysis is not run in this isolated experiment set
    # resnet_result = analyze_resnet_block_noise()
    # results.append(resnet_result)

    return pd.DataFrame(results)

# Run only PTB-XL experiments
results_df_ptbxl = run_ptbxl_experiments()

#@title # 8. Save and Display Results (PTB-XL Only)
print("\n\n--- PTB-XL Experiments Complete ---")
print("PTB-XL Accuracy Results:")

# Display the results DataFrame
display(results_df_ptbxl)

# Save the DataFrame to a CSV file (optional, can merge later)
output_filename_ptbxl = 'split_learning_accuracy_results_ptbxl_only.csv'
results_df_ptbxl.to_csv(output_filename_ptbxl, index=False)
print(f"\nPTB-XL Results saved to {output_filename_ptbxl}")


--- Skipping PTB_XL_CNN experiments due to data loading failure. ---


--- PTB-XL Experiments Complete ---
PTB-XL Accuracy Results:


""



PTB-XL Results saved to split_learning_accuracy_results_ptbxl_only.csv


In [38]:
#@title # PTB-XL Experiment: Full Workflow (Corrected)

# ==============================================================================
# 1. SETUP AND IMPORTS
# ==============================================================================
# This cell contains all necessary code to run the PTB-XL experiment.
# It will download the data, define the model, and run the training loops.

print("--- Step 1: Installing and setting up dependencies ---")
!pip install wfdb kagglehub -q
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import random
import kagglehub
import ast
import wfdb # Library for reading waveform data
from IPython.display import display

# Set up the device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a seed for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

# --- Experiment Configuration ---
HE_NOISE_SIGMA = 1e-4
EPOCHS = 10 # Increase for final, more accurate results
LEARNING_RATE = 0.001
BATCH_SIZE = 64

# ==============================================================================
# 2. MODEL DEFINITION
# ==============================================================================
print("\n--- Step 2: Defining Model Architecture ---")

class SimulatedHENoise(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, x):
        if self.training and self.sigma > 0:
            return x + torch.randn_like(x) * self.sigma
        return x

class SplitModelBase(nn.Module):
    def __init__(self, cut_layer):
        super().__init__()
        self.cut_layer = cut_layer
        self.noise_injector = SimulatedHENoise(HE_NOISE_SIGMA)
        self.layers_list = nn.ModuleList()

    def forward(self, x):
        for i, layer in enumerate(self.layers_list):
            x = layer(x)
            # Inject noise if the layer is on the server
            if i < self.cut_layer:
                x = self.noise_injector(x)
        return x

class PTB_XL_CNN(SplitModelBase):
    def __init__(self, cut_layer, he_noise_sigma=HE_NOISE_SIGMA):
        super().__init__(cut_layer)
        self.layers_list.extend([
            nn.Conv1d(in_channels=12, out_channels=16, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(8 * 250, 2000),
            nn.Linear(2000, 5)
        ])
print("Model architecture defined.")

# ==============================================================================
# 3. KAGGLEHUB DOWNLOAD
# ==============================================================================
print("\n--- Step 3: Downloading PTB-XL dataset via KaggleHub ---")
try:
    download_path = kagglehub.dataset_download("khyeh0719/ptb-xl-dataset")
    sub_dirs = [d for d in os.listdir(download_path) if os.path.isdir(os.path.join(download_path, d))]
    full_data_path = os.path.join(download_path, sub_dirs[0]) if sub_dirs else download_path
    print(f"Dataset successfully located at: {full_data_path}")
except Exception as e:
    print(f"KaggleHub download failed. Error: {e}")
    full_data_path = None

# ==============================================================================
# 4. DATA LOADING FUNCTION (WITH WFDB READER)
# ==============================================================================
def get_ptbxl_loaders(dataset_path, batch_size=BATCH_SIZE):
    print("\n--- Step 4: Loading and preprocessing PTB-XL data ---")
    db_file = os.path.join(dataset_path, 'ptbxl_database.csv')
    scp_file = os.path.join(dataset_path, 'scp_statements.csv')

    Y = pd.read_csv(db_file, index_col='ecg_id')
    Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
    scp_statements = pd.read_csv(scp_file, index_col=0)
    scp_statements = scp_statements[scp_statements.diagnostic == 1]

    def aggregate_diagnostic(y_dic):
        tmp = []
        for key in y_dic.keys():
            if key in scp_statements.index:
                tmp.append(scp_statements.loc[key].diagnostic_class)
        return list(set(tmp)) if tmp else []

    Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
    Y = Y[Y.diagnostic_superclass.apply(len) > 0]

    train_df = Y[Y.strat_fold < 10]
    test_df = Y[Y.strat_fold == 10]

    # --- CORRECTED SIGNAL LOADING WITH WFDB ---
    def load_signals(df_meta, base_path):
        X_data, y_labels = [], []
        for _, row in tqdm(df_meta.iterrows(), total=len(df_meta), desc="Loading WFDB signals"):
            # Construct the path to the record WITHOUT the file extension
            record_path = os.path.join(base_path, row.filename_lr)
            try:
                # wfdb.rdsamp reads both .dat and .hea files
                signal, _ = wfdb.rdsamp(record_path)
                X_data.append(signal)
                y_labels.append(row.diagnostic_superclass[0])
            except Exception as e:
                print(f"Warning: Could not read record {record_path}. Error: {e}")
        return np.array(X_data), y_labels

    X_train, y_train_list = load_signals(train_df, dataset_path)
    X_test, y_test_list = load_signals(test_df, dataset_path)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Fatal Error: No data was loaded. Aborting.")
        return None, None

    # Preprocessing: Transpose from (Length, Channels) to (Channels, Length)
    X_train = X_train.transpose(0, 2, 1)
    X_test = X_test.transpose(0, 2, 1)

    # Label encoding
    all_labels = sorted(list(set(y_train_list + y_test_list)))
    superclass_map = {label: i for i, label in enumerate(all_labels)}
    y_train = np.array([superclass_map[label] for label in y_train_list])
    y_test = np.array([superclass_map[label] for label in y_test_list])
    print(f"Data loaded. Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ==============================================================================
# 5. TRAINING AND EVALUATION LOOPS
# ==============================================================================
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate(model, dataloader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# ==============================================================================
# 6. RUN THE EXPERIMENT
# ==============================================================================
if full_data_path:
    print("\n--- Step 5: Initializing PTB-XL Experiment ---")
    train_loader_ptbxl, test_loader_ptbxl = get_ptbxl_loaders(full_data_path)

    if train_loader_ptbxl and test_loader_ptbxl:
        results = []
        criterion = nn.CrossEntropyLoss()
        total_layers = len(PTB_XL_CNN(0).layers_list)

        for cut in tqdm(range(total_layers + 1), desc="Testing PTB_XL_CNN cuts"):
            print(f"\n--- Training PTB_XL_CNN with cut_layer = {cut} ---")
            model = PTB_XL_CNN(cut_layer=cut).to(device)
            optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

            for epoch in range(EPOCHS):
                train_one_epoch(model, train_loader_ptbxl, criterion, optimizer, device)
                acc = evaluate(model, test_loader_ptbxl, device)
                print(f"Epoch {epoch+1}/{EPOCHS}, Test Accuracy: {acc:.2f}%")

            final_accuracy = evaluate(model, test_loader_ptbxl, device)
            print(f"Final Accuracy for PTB_XL_CNN (cut={cut}): {final_accuracy:.2f}%")
            results.append({'model': 'PTB_XL_CNN', 'cut_layer': cut, 'accuracy': final_accuracy})

        # Display and Save Results
        ptbxl_results_df = pd.DataFrame(results)
        print("\n\n--- PTB-XL Experiment Complete ---")
        display(ptbxl_results_df)
        ptbxl_results_df.to_csv('ptbxl_split_learning_results.csv', index=False)
        print("\nResults saved to ptbxl_split_learning_results.csv")
    else:
        print("\nCould not run PTB-XL experiment due to data loading failure.")
else:
    print("\nCould not run PTB-XL experiment due to dataset download failure.")

--- Step 1: Installing and setting up dependencies ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 117.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
Using device: cuda

--- Step 2: Defining Model Architecture ---
Model architecture defined.

--- Step 3: Downloading PTB-XL dataset via KaggleHub ---
Dataset successfully located at: /kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly

Loading WFDB signals:   0%|          | 0/19267 [00:00<?, ?it/s]

Loading WFDB signals:   0%|          | 0/2163 [00:00<?, ?it/s]

Data loaded. Train shape: (19267, 12, 1000), Test shape: (2163, 12, 1000)


Testing PTB_XL_CNN cuts:   0%|          | 0/10 [00:00<?, ?it/s]


--- Training PTB_XL_CNN with cut_layer = 0 ---
Epoch 1/10, Test Accuracy: 57.14%
Epoch 2/10, Test Accuracy: 62.14%
Epoch 3/10, Test Accuracy: 63.71%
Epoch 4/10, Test Accuracy: 63.38%
Epoch 5/10, Test Accuracy: 64.03%
Epoch 6/10, Test Accuracy: 63.80%
Epoch 7/10, Test Accuracy: 64.26%
Epoch 8/10, Test Accuracy: 63.71%
Epoch 9/10, Test Accuracy: 63.48%
Epoch 10/10, Test Accuracy: 62.83%
Final Accuracy for PTB_XL_CNN (cut=0): 62.83%

--- Training PTB_XL_CNN with cut_layer = 1 ---
Epoch 1/10, Test Accuracy: 57.51%
Epoch 2/10, Test Accuracy: 62.18%
Epoch 3/10, Test Accuracy: 61.90%
Epoch 4/10, Test Accuracy: 62.60%
Epoch 5/10, Test Accuracy: 63.80%
Epoch 6/10, Test Accuracy: 64.08%
Epoch 7/10, Test Accuracy: 63.20%
Epoch 8/10, Test Accuracy: 63.29%
Epoch 9/10, Test Accuracy: 63.52%
Epoch 10/10, Test Accuracy: 63.06%
Final Accuracy for PTB_XL_CNN (cut=1): 63.06%

--- Training PTB_XL_CNN with cut_layer = 2 ---
Epoch 1/10, Test Accuracy: 57.61%
Epoch 2/10, Test Accuracy: 61.63%
Epoch 3/10, Te

,model,cut_layer,accuracy
0,PTB_XL_CNN,0,62.829404
1,PTB_XL_CNN,1,63.060564
2,PTB_XL_CNN,2,62.783172
3,PTB_XL_CNN,3,62.690707
4,PTB_XL_CNN,4,63.522885
5,PTB_XL_CNN,5,64.956080
6,PTB_XL_CNN,6,62.274619
7,PTB_XL_CNN,7,62.135922
8,PTB_XL_CNN,8,63.985206
9,PTB_XL_CNN,9,63.106796



Results saved to ptbxl_split_learning_results.csv
